## Scraping movies from IMDbTop 1000

IMDB ( viết tắt của Internet Movie Database) là cơ sở dữ liệu trực tuyến chứa thông tin liên quan đến phim, phim truyền hình, video gia đình, trò chơi điện tử và nội dung phát trực tuyến – bao gồm dàn diễn viên, đội ngũ sản xuất và tiểu sử cá nhân, tóm tắt cốt truyện, câu đố, xếp hạng, đánh giá của người hâm mộ và giới phê bình. IMDb bắt đầu như một cơ sở dữ liệu phim do người hâm mộ điều hành trên nhóm Usenet "rec.arts.movies" vào năm 1990 và chuyển sang Web vào năm 1993. Hiện tại, IMDb.com, Inc. , một công ty con của Amazon , sở hữu và điều hành.

![](1.png)

Chúng ta sẽ lấy top 1000 bộ phim (dựa trên điểm rating) của IMDb bằng thư viện Beautiful Soup (BS4)
Beautiful Soup (BS4) là một thư viện phân tích cú pháp có thể sử dụng các parsers khác nhau từ đó có thể trích xuất dữ liệu từ các tài liệu HTML và XML một cách dễ dàng. Sau khi phân tích các HTML cũng như XML đầu vào, Beautiful Soup cho phép chúng ta dễ dàng di chuyển, tìm kiếm, thay đổi cũng như trích xuất dữ liệu từ cây cú pháp. Cú pháp rõ ràng linh hoạt và sự mạnh mẽ các lý do khiến Beautiful Soup trở thành một trong những công cụ phổ biến nhất cho việc thu thập dữ liệu web.\ 

**Sau đây là chi tiết cách làm.**

### Sử dụng thư viện BeautifulSoup để lấy dữ liệu 

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [102]:
headers = {'Accept-Language': 'en-US, en;q=0.5'}

### Tạo danh sách các url 

In [103]:
url_list = ['https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating']

for i in range(101, 1001, 100):
    url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={}&ref_=adv_nxt'
    formated_url = url.format(str(i))
    url_list.append(formated_url)
url_list

['https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=101&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=201&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=301&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=401&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=501&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=601&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=701&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=801&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?g

### Hàm lấy dữ liệu từ url cho trước và trả về một list

In [211]:
def get_movies_from_url(url, data):
    
    html_text = requests.get(url, headers = headers).text
    soup = BeautifulSoup(html_text, 'lxml')
    movies = soup.find_all('div', class_ = 'lister-item mode-advanced') 
    
    for movie in movies:
        
        link_imdb = 'https://www.imdb.com' + movie.find('h3', class_ = 'lister-item-header').a['href']
        
        link_poster = movie.find('div', class_ = 'lister-item-image float-left').a.img['loadlate']
        
        title = movie.find('h3', class_ = 'lister-item-header').a.text
        
        year = movie.find('span', class_ = 'lister-item-year text-muted unbold').text
        
        certificate = movie.find('span', class_ = 'certificate').text.strip() if movie.find('span', class_ = 'certificate') else None
        
        runtime = movie.find('span', class_ = 'runtime').text.strip() 
        
        genre = movie.find('span', class_ = 'genre').text.strip()
        
        rating = movie.find('div',class_= 'inline-block ratings-imdb-rating').strong.text
        
        meta_score = movie.find('span', class_ = 'metascore favorable').text.strip() if movie.find('span', class_ = 'metascore favorable') else None
        
        overview = movie.find_all('p', class_= 'text-muted')[1].text.strip()
        
        ds = movie.find('p', class_ = '').find_all('a')
        director = ds[0].text
        star_1 = ds[1].text
        star_2 = ds[2].text
        star_3 = ds[3].text
        star_4 = ds[4].text
        
        nv = movie.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        gross = nv[1].text if (len(nv) > 1) and (nv[1].text[0] != '#') else None  
        
        data.append([link_imdb, link_poster, title, year, certificate, 
                     runtime, genre, rating, meta_score, overview, 
                     director, star_1, star_2, star_3, star_4, vote, gross])

### Lấy dữ liệu từ các url trong danh sách và lưu vào list

In [110]:
data = []
for url in url_list:
    time.sleep(10)
    get_movies_from_url(url, data)

### Chuyển sang dataframe

In [178]:
df = pd.DataFrame(data, columns= ["link_imdb", "link_poster", "title", "year", "certificate"
                                  , "runtime", "genre", "rating", "meta_4score", "overview"
                                  , "director","star_1", "star_2", "star_3", "star_4", "vote", "gross"])

### Xuất ra file csv

In [179]:
df.to_csv('imdb_top_1000_movies.csv', index = False)